In [1]:
import time
import pyspark
import numpy as np
import linecache
import csv
from operator import add
from pyspark.sql import SQLContext


In [2]:
file_name = './data/News_Final_1.csv'
sc = pyspark.SparkContext(appName='hw2-3')
text_file = sc.textFile(file_name)
sqlContext=SQLContext(sc)
news_data=sqlContext.read.format('csv')\
                    .option("header", "true")\
                    .option("inferschema", "true")\
                    .option("mode", "DROPMALFORMED")\
                    .load(file_name)
news_data.printSchema()

# news_data=news_data.withColumn('SentimentTitle', news_data['SentimentTitle'].cast('double'))
# news_data=news_data.withColumn('SentimentHeadline', news_data['SentimentHeadline'].cast('double'))
# news_data.show(5)



root
 |-- IDLink: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Headline: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- PublishDate: timestamp (nullable = true)
 |-- SentimentTitle: double (nullable = true)
 |-- SentimentHeadline: double (nullable = true)
 |-- Facebook: integer (nullable = true)
 |-- GooglePlus: integer (nullable = true)
 |-- LinkedIn: integer (nullable = true)



In [ ]:
SentimentScore=news_data.select('Topic', 'SentimentTitle', 'SentimentHeadline').rdd.map(list).map(lambda x:(x[0], [x[1], x[2], 1])).reduceByKey(lambda x, y:[x[0]+y[0], x[1]+y[1], x[2]+y[2]])
SentimentScore_sum=SentimentScore.sortByKey(False).map(lambda x:[x[0], x[1][0], x[1][1]]).collect()
SentimentScore_average=SentimentScore.sortByKey(False).map(lambda x:[x[0], x[1][0]/x[1][2], x[1][1]/x[1][2]]).collect()

out=open('output/SentimentScore.txt','w')
out.write('SentimentScore_sum\n')
print(SentimentScore_sum)
for i in range(len(SentimentScore_sum)):
    out.write(str(SentimentScore_sum[i]) + '\n')
out.write('\nSentimentScore_average\n')
for i in range(len(SentimentScore_average)):
    out.write(str(SentimentScore_average[i]) + '\n')
out.close()

In [ ]:
# with open(file_name) as file:
#     array = file.readlines()
#     array = [row.split(',') for row in array]
# for elm in array:
#     print(elm)

In [ ]:
# for i in range(len(array)):
#     print(array[i][4])

In [ ]:
sc.stop()

In [ ]:
# df = sqlContext.read.format("csv").option("header","true").option("inferSchema","true").load(file_name)

# df.select(df['Topic'], df['SentimentTitle'], df['SentimentHeadline']).show()

#df.show()


# parts = text_file.map(lambda l:l.split(","))
# people = parts.map(lambda p:(p[6],p[6].strip()))

# schemaString = "SentimentTitle SentimentHeadline"

# fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
# schema = StructType(fields)

# schemaPeople = sqlContext.createDataFrame(people, schema)

# schemaPeople.registerTempTable("people")

# results = sqlContext.sql("SELECT SentimentTitle from people")

# STs = results.rdd.map(lambda p:"SentimentTitle: " + p.SentimentTitle)

# for st in STs.collect():
#     print(st)